# Install Dependencies

In [2]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd

import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
from pprint import pprint

%matplotlib inline     
sns.set(color_codes=True)

# Grab Wifire Simulations Run Data 

In [1]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
# FIX THIS

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'
# path = '00/16/run_001676e4-ed7a-42e4-abc5-9c6f1dc122df'
# with fs.open( bucket + '/' + path + '/' + name + '/.zattrs') as f:
#         data=json.load(f)

root = list(fs.ls(bucket))
simulation_paths = []

NameError: name 's3fs' is not defined

In [11]:
def get_paths(bucket):
    global fs
    paths = fs.ls(bucket)
    for path in tqdm(paths[1:2]):
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

In [12]:
get_paths(bucket)
len(simulation_paths)

PermissionError: The request signature we calculated does not match the signature you provided. Check your key and signing method.

# Data Exploration

In [10]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    # 'dz',
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df(count):
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys())
    i = 0
    for p in tqdm(simulation_paths[:count]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
                # print("list:",[KEEP_ATTRIBUTES[i](data) for i in KEEP_ATTRIBUTES.keys()])
            
            row = []
            for k,expr in KEEP_ATTRIBUTES.items():
                try:
                    value = KEEP_ATTRIBUTES[k](data)
                except KeyError:
                    value = None
                row.append(value)
            row[0] = p
            df.loc[i] = row
            i+=1
        except FileNotFoundError as e:
            filenotfound.append(p)
            print("FileNotFound error on path {", e,"}")
    return df

pd.set_option('display.max_columns', None)

df = get_df(20)

NameError: name 'simulation_paths' is not defined

In [223]:
from datetime import datetime
import iso8601

date_str = '2022-07-13T00:57:37.108269-07:00'
iso8601.parse_date(date_str)

datetime.datetime(2022, 7, 13, 0, 57, 37, 108269, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), '-07:00'))

In [225]:
path = 'burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-9635bdb61ec5'
with fs.open(path + '/' + name + '/.zattrs') as f:
    data=json.load(f)
    # print("list:",[KEEP_ATTRIBUTES[i](data) for i in KEEP_ATTRIBUTES.keys()])
    print(data)

PermissionError: Forbidden

In [224]:
# these are the top level directories, all 200
p = 'burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-021cf9ee4e9c'
with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
pprint(data,compact=True,depth=2)
# print(data["resolution"])
# pprint(list(data.keys()))

PermissionError: Forbidden

# Next Step

In [12]:
# pandas group by version in dataframe

# caerful we have ign oor fuel object
# only look for ones that say run

In [13]:
# scan top directory 
# ls bucket but max step is 01